In [1]:
import pandas as pd
import xlrd
import numpy as np 
import matplotlib
import matplotlib.pylab as plt
from sklearn.decomposition import PCA
from sklearn import linear_model
from pandas.tools.plotting import scatter_matrix
import seaborn as sns
import codecs
import csv
from scipy import stats
import     statsmodels.api as sm
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC

from pandas.tools.plotting import scatter_matrix
from pandas.tools.plotting import lag_plot
from pandas.tools.plotting import autocorrelation_plot
from math import sqrt
from sklearn.metrics import mean_squared_error

In [2]:
#描画するグラフのスタイルを指定。日本語表示可なフォントとフォントサイズ、背景色を白に指定。
sns.set(font="IPAexGothic",style="whitegrid", font_scale=0.2)

In [3]:
cd C://Users//\A. Ichinose//Desktop/hd_jpynb

C:\Users\A. Ichinose\Desktop\hd_jpynb


In [4]:
#sip2015を開く
book="sip2015_2.xlsx"
#読み込むシート名
sheet="All"
#エクセルファイルをPython上で開く
EXL = pd.ExcelFile(book)
df = EXL.parse(sheet,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,804,805,806,807,808,809,810,811,812,813
0,NaN,NaN,NaN,NaN,NaN,NaN,身長・体重,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date,ID,Sex,Age,MEQ,MEQ補正,Ht,Wt,%fat,BMI,...,Div_lr_a46_l,Div_lr_a46_r,Div_lr_a47_l,Div_lr_a47_r,Div_lr_a06_l,Div_lr_a06_r,Div_lr_a08_l,Div_lr_a08_r,Div_lr_a09_l,Div_lr_a09_r
2,日,単位,-,才,NaN,NaN,ｃｍ,ｋｇ,%,-,...,mmol×mm,mmol×mm,mmol×mm,mmol×mm,mmol×mm,mmol×mm,mmol×mm,mmol×mm,mmol×mm,mmol×mm
3,20150228,1,M,63,69,60.6457,176.3,70.1,24.5,22.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20150228,2,M,62,67,58.9969,169.8,62.9,24.1,21.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#index=0と2の行を削除
df1=df.drop(index=[0,2])
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,804,805,806,807,808,809,810,811,812,813
1,Date,ID,Sex,Age,MEQ,MEQ補正,Ht,Wt,%fat,BMI,...,Div_lr_a46_l,Div_lr_a46_r,Div_lr_a47_l,Div_lr_a47_r,Div_lr_a06_l,Div_lr_a06_r,Div_lr_a08_l,Div_lr_a08_r,Div_lr_a09_l,Div_lr_a09_r
3,20150228,1,M,63,69,60.6457,176.3,70.1,24.5,22.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20150228,2,M,62,67,58.9969,169.8,62.9,24.1,21.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,20150228,4,M,66,73,63.5921,154.3,57.6,29.7,24.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#インデックスを振り直し
df2 =df1.reset_index(drop=True)
df2.head(10)

,0,1,2,3,4,5,6,7,8,9,...,804,805,806,807,808,809,810,811,812,813
0,Date,ID,Sex,Age,MEQ,MEQ補正,Ht,Wt,%fat,BMI,...,Div_lr_a46_l,Div_lr_a46_r,Div_lr_a47_l,Div_lr_a47_r,Div_lr_a06_l,Div_lr_a06_r,Div_lr_a08_l,Div_lr_a08_r,Div_lr_a09_l,Div_lr_a09_r
1,20150228,1,M,63,69,60.6457,176.3,70.1,24.5,22.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20150228,2,M,62,67,58.9969,169.8,62.9,24.1,21.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20150228,4,M,66,73,63.5921,154.3,57.6,29.7,24.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20150311,5,M,63,62,53.6457,175.3,67.1,25,21.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,20150311,6,M,71,44,32.8361,165,53.5,16,19.7,...,-0.297049,1.22069,-0.492534,0.97885,0.446842,0.852569,0.388313,0.738717,0.223331,0.392758
7,20150311,7,M,67,63,53.2409,171.8,62.6,27.2,21.2,...,-0.135845,-0.303859,-0.549529,-0.219489,0.844174,-0.535215,-0.286748,-0.341105,-0.409601,-0.650827
8,20150311,8,F,69,63,52.5385,148,54.6,38.1,24.9,...,-0.804605,-0.945262,-0.309072,-0.275152,-0.29145,0.553102,-0.931123,-0.605906,-0.685734,-0.283366
9,20150311,9,F,65,58,48.9433,155,57.1,33.8,23.8,...,-0.25892,-0.64329,-0.0165905,-0.0579231,-0.616789,0.0572832,-0.114324,-0.519159,-0.0346268,-0.0206682


In [7]:
df2.shape

(212, 814)

In [15]:
#for_vf_estimationを開く
book3="for_vf_estimation.xlsx"
#読み込むシート名
sheet="Sheet1"
#エクセルファイルをPython上で開く
EXL = pd.ExcelFile(book3)
dfvf = EXL.parse(sheet,header=None)
dfvf.head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,L4,L4,NaN,NaN
1,Date,ID,Sex,Age,Ht,Wt,%fat,BMI,FAL,UAL,...,凸-W,W-凹,凸-凹,凸-W臍,W-凹臍,凸臍-凹臍,VF,SF,VFE,SFE
2,日,単位,-,才,ｃｍ,ｋｇ,%,-,ｃｍ,ｃｍ,...,ｃｍ,ｃｍ,ｃｍ,ｃｍ,ｃｍ,ｃｍ,cm2,cm2,cm2,cm2
3,20150228,001,M,63,176.3,70.1,24.5,22.5,26,32.5,...,NaN,NaN,NaN,NaN,NaN,NaN,125.73,80.77,106.694,78.6666
4,20150228,002,M,62,169.8,62.9,24.1,21.8,21.5,26.5,...,NaN,NaN,NaN,NaN,NaN,NaN,64.61,68.97,108.681,86.6606


In [16]:
#index=0と2の行を削除
dfvf1=dfvf.drop(index=[0,2])
dfvf1.head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
1,Date,ID,Sex,Age,Ht,Wt,%fat,BMI,FAL,UAL,...,凸-W,W-凹,凸-凹,凸-W臍,W-凹臍,凸臍-凹臍,VF,SF,VFE,SFE
3,20150228,001,M,63,176.3,70.1,24.5,22.5,26,32.5,...,NaN,NaN,NaN,NaN,NaN,NaN,125.73,80.77,106.694,78.6666
4,20150228,002,M,62,169.8,62.9,24.1,21.8,21.5,26.5,...,NaN,NaN,NaN,NaN,NaN,NaN,64.61,68.97,108.681,86.6606
5,欠番,003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,20150228,004,M,66,154.3,57.6,29.7,24.1,20.5,28,...,NaN,NaN,NaN,NaN,NaN,NaN,104.56,118.68,110.153,102.207


In [17]:
#インデックスを振り直し
dfvf2 =dfvf1.reset_index(drop=True)
dfvf2.head(10)

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,Date,ID,Sex,Age,Ht,Wt,%fat,BMI,FAL,UAL,...,凸-W,W-凹,凸-凹,凸-W臍,W-凹臍,凸臍-凹臍,VF,SF,VFE,SFE
1,20150228,001,M,63,176.3,70.1,24.5,22.5,26,32.5,...,NaN,NaN,NaN,NaN,NaN,NaN,125.73,80.77,106.694,78.6666
2,20150228,002,M,62,169.8,62.9,24.1,21.8,21.5,26.5,...,NaN,NaN,NaN,NaN,NaN,NaN,64.61,68.97,108.681,86.6606
3,欠番,003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20150228,004,M,66,154.3,57.6,29.7,24.1,20.5,28,...,NaN,NaN,NaN,NaN,NaN,NaN,104.56,118.68,110.153,102.207
5,20150311,005,M,63,175.3,67.1,25,21.8,24,33.5,...,NaN,NaN,NaN,NaN,NaN,NaN,106.07,111.42,114.858,90.3466
6,20150311,006,M,71,165,53.5,16,19.7,21.5,32.5,...,NaN,NaN,NaN,NaN,NaN,NaN,43.35,62.88,95.508,29.063
7,20150311,007,M,67,171.8,62.6,27.2,21.2,23.5,34.5,...,NaN,NaN,NaN,NaN,NaN,NaN,152.98,88.31,125.436,91.8146
8,20150311,008,F,69,148,54.6,38.1,24.9,20.5,27.5,...,NaN,NaN,NaN,NaN,NaN,NaN,101.78,153.6,77.023,176.72
9,20150311,009,F,65,155,57.1,33.8,23.8,20.5,27.5,...,NaN,NaN,NaN,NaN,NaN,NaN,79.66,135.71,75.741,173.683


In [21]:
#index0の行をカラムとして設定
dfvf2.columns=dfvf2.iloc[0]

In [23]:
dfvf3=dfvf2.loc[:,['ID','凸-凹','W臍']]
dfvf3.head()

,ID,凸-凹,W臍
0,ID,凸-凹,W臍
1,001,NaN,84
2,002,NaN,85
3,003,NaN,NaN
4,004,NaN,84


In [24]:
#sip2015とfor_vf...の3項目抽出済みdfvf3をIDを軸としてマージ
df_m=pd.merge(df2, dfvf3, on='ID')

In [25]:
df_m.head()

,Date,ID,Sex,Age,MEQ,MEQ補正,Ht,Wt,%fat,BMI,...,Div_lr_a47_l,Div_lr_a47_r,Div_lr_a06_l,Div_lr_a06_r,Div_lr_a08_l,Div_lr_a08_r,Div_lr_a09_l,Div_lr_a09_r,凸-凹,W臍
0,Date,ID,Sex,Age,MEQ,MEQ補正,Ht,Wt,%fat,BMI,...,Div_lr_a47_l,Div_lr_a47_r,Div_lr_a06_l,Div_lr_a06_r,Div_lr_a08_l,Div_lr_a08_r,Div_lr_a09_l,Div_lr_a09_r,凸-凹,W臍
1,20151112,130,M,45,71,68.9673,175.5,65.1,20.6,21.1,...,-0.51425,0.600528,-0.169703,-0.369655,-0.459354,-0.702086,0.0804178,-0.507337,5.3,81.5
2,20151112,131,F,61,54,46.3481,150.8,52,30.6,22.8,...,0.0971377,0.440072,-0.0315765,-0.350811,0.0142739,-0.0813108,-0.313489,-0.185899,1.6,85.2
3,20151112,132,F,54,66,60.8065,157.4,57.6,33.5,23.2,...,-1.0651,-0.32377,-0.485059,0.288434,0.661353,0.429745,0.48942,0.142842,5.7,90.6
4,20151112,133,F,73,49,37.1337,146.7,37.8,32.6,17.6,...,0.0318959,0.76087,0.541476,-0.135692,0.0571998,-0.534198,0.149955,-0.0512257,3.7,69.2


In [26]:
df_m.shape

(62, 816)

In [27]:
dfvf3.shape

(261, 3)

In [28]:
#df_mの行数が少ないので全貌をチェック
#行を省略せずに表示(500行まで表示)
pd.set_option('display.max_rows', 500)
df_m.head(500)

,Date,ID,Sex,Age,MEQ,MEQ補正,Ht,Wt,%fat,BMI,...,Div_lr_a47_l,Div_lr_a47_r,Div_lr_a06_l,Div_lr_a06_r,Div_lr_a08_l,Div_lr_a08_r,Div_lr_a09_l,Div_lr_a09_r,凸-凹,W臍
0,Date,ID,Sex,Age,MEQ,MEQ補正,Ht,Wt,%fat,BMI,...,Div_lr_a47_l,Div_lr_a47_r,Div_lr_a06_l,Div_lr_a06_r,Div_lr_a08_l,Div_lr_a08_r,Div_lr_a09_l,Div_lr_a09_r,凸-凹,W臍
1,20151112,130,M,45,71,68.9673,175.5,65.1,20.6,21.1,...,-0.51425,0.600528,-0.169703,-0.369655,-0.459354,-0.702086,0.0804178,-0.507337,5.3,81.5
2,20151112,131,F,61,54,46.3481,150.8,52,30.6,22.8,...,0.0971377,0.440072,-0.0315765,-0.350811,0.0142739,-0.0813108,-0.313489,-0.185899,1.6,85.2
3,20151112,132,F,54,66,60.8065,157.4,57.6,33.5,23.2,...,-1.0651,-0.32377,-0.485059,0.288434,0.661353,0.429745,0.48942,0.142842,5.7,90.6
4,20151112,133,F,73,49,37.1337,146.7,37.8,32.6,17.6,...,0.0318959,0.76087,0.541476,-0.135692,0.0571998,-0.534198,0.149955,-0.0512257,3.7,69.2
5,20151112,134,F,52,48,43.5089,161.5,66.6,35.9,25.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.3,98.4
6,20151112,135,F,49,70,66.5625,158.1,49,25.1,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.1,72.7
7,20151112,136,M,57,61,54.7529,168.6,65.6,21.1,23.1,...,-0.423367,0.366658,-0.0836073,-0.264147,0.159171,0.331622,0.372759,0.263772,7.1,81.4
8,20151112,137,M,50,59,55.2113,171,65.2,21.3,23.3,...,-0.36282,0.736055,0.414883,0.197341,-0.0895193,-0.128388,0.42197,-0.165482,8.1,84.5
9,20151116,138,M,48,46,42.9137,165.6,79.6,26.6,29.1,...,0.146686,-0.114797,0.458038,0.273036,0.322241,0.390784,0.649067,0.61145,7.4,97


In [ ]:
#外れ値を除去
tuikaM2=tuikaM1.drop(index=[81,3,28])

In [8]:
#index0の行をカラムとして設定
df2.columns=df2.iloc[0]

In [9]:
#石川さんの外れ値を削除
df3=df2.drop(index=[137,88,35,36,58,139,38,141,40,54,98,101])

In [12]:
df3=df3.dropna(axis=0, how='all')
df3=df3.dropna(axis=0, subset=["エネルギー"])
df3=df3.dropna(axis=0, subset=['VF'])
df3=df3.dropna(axis=0, subset=['SF'])
df3=df3.dropna(axis=0, subset=['中性脂肪'])
df3=df3.dropna(axis=0, subset=['ＨＤＬ－Ｃ'])
df3=df3.dropna(axis=0, subset=['ＬＤＬ－Ｃ'])
df3=df3.dropna(axis=0, subset=['血糖（空腹時）'])
df3=df3.dropna(axis=0, subset=['収縮期'])
df3=df3.dropna(axis=0, subset=['拡張期'])
df3=df3.dropna(axis=0, subset=['MEQ'])
df3=df3.dropna(axis=0, subset=['LPA'])
df3=df3.dropna(axis=0, subset=['緊張・不安'])
df3=df3.dropna(axis=0, subset=['朝食時刻'])
df3=df3.dropna(axis=0, subset=['昼食時刻'])
df3=df3.dropna(axis=0, subset=["Age"])
df3=df3.dropna(axis=0, subset=["W"])
df3=df3.dropna(axis=0, subset=["W/H"])

In [13]:
df3.shape

(164, 814)